In [70]:
import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
#pip3 install progress progressbar2 alive-progress tqdm

In [64]:
df_stu, meta_stu = pyreadstat.read_sav('data/CY08MSP_STU_QQQ_HUN.sav')

In [65]:
import os
with open('data/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [66]:
df_stu['STUD_MATH'] = 0
df_stu['STUD_READ'] = 0
df_stu['STUD_SCIE'] = 0

cols = utils.meta2docs(meta_stu, excluded=['test','CNTRYID'])

cols.append( Document(
                    page_content= 'Student\'s score in Mathematics',
                    metadata={"year": 2022, "original_col_name": 'STUD_MATH'},
                ))

cols.append( Document(
                    page_content= 'Student\'s score in Reading',
                    metadata={"year": 2022, "original_col_name": 'STUD_READ'},
                ))

cols.append( Document(
                    page_content= 'Student\'s score in Science',
                    metadata={"year": 2022, "original_col_name": 'STUD_SCIE'},
                ))
#drop empty columns
df_stu = df_stu.dropna(axis=1, how='all')

cols2 = []
for col in cols:
    if col.metadata['original_col_name'] in df_stu.columns:
        cols2.append(col)
cols = cols2

cols_vectorstore = FAISS.from_documents(cols, OpenAIEmbeddings())

cols_retriever = cols_vectorstore.as_retriever()

In [67]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_community.cache import SQLiteCache


llm = ChatOpenAI(temperature=0)
set_llm_cache(SQLiteCache(database_path=".langchain.db"))


In [68]:
qs = utils.load_questions('questions_c108_20240901_193906.txt')

In [76]:
res = utils.execute_tests(llm, qs, df_stu, meta_stu, cols_retriever)

  0%|          | 0/108 [00:00<?, ?it/s]

[Error] Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).
[Error] Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).
[Error] name 'mean_squared_error' is not defined
[Error] '(' was never closed (<string>, line 7)
[Error] name 'mean_squared_error' is not defined
[Error] name 'loc' is not defined
[Error] name 'loc' is not defined
[Error] name 'loc' is not defined
[Error] name 'loc' is not defined
[Error] name 'loc' is not defined
[Error] name 'loc' is not defined
[Error] name 'loc' is not defined
[Error] name 'loc' is not defined
[Error] name 'loc' is not defined
[Error] name 'loc' is not defined
[Error] name 'loc' is not defined
[Error] name 'loc' is not defined
[Error] name 'loc' is not defined
[Error] name 'loc' is not defined
[Error] name 'loc' is not defined
[Error] name 'loc' is not defined
[Error] name 'loc' is not defined
[Error] name 'loc' is not defined
[Error] name 'loc' is not defined
[Error] name 'loc' is not 

In [72]:
pdres = pd.DataFrame(res)
len(pdres[pdres['result'].isnull()])


35

In [73]:
pdres

,question,result,inter,used_columns,hint_cols,code
0,What is the relation between social background...,"(47.26859645731944, Linear Regression)",The linear regression coefficient for the rela...,"[ESCS, STUD_MATH]","[(ESCS, Index of economic, social and cultural...","def run(data):\n data = data[['ESCS', 'STUD..."
1,Can you provide the linear regression coeffici...,"(([47.216271605008814], 1.1168192023120016), L...",The linear regression coefficient for the rela...,"[STUD_MATH, ESCS, STUD_MATH, STUD_MATH]","[(STUD_MATH, Student's score in Mathematics), ...","def run(data):\n data = data[['STUD_MATH', ..."
2,I am interested in knowing the linear regressi...,None,"The result ""None"" likely means that there was ...","[STUD_MATH, COBN_F, COBN_M, ESCS]","[(STUD_MATH, Student's score in Mathematics), ...","def run(data):\n data = data[['STUD_MATH', ..."
3,Could you share the linear regression coeffici...,"(([47.216271605008814], 1.1168192023120016), L...",The linear regression coefficient for the corr...,"[STUD_MATH, ESCS, STUD_MATH, STUD_MATH]","[(STUD_MATH, Student's score in Mathematics), ...","def run(data):\n data = data[['STUD_MATH', ..."
4,I would like to see the linear regression coef...,"((47.21627160500879, 1.1168192023120016), Line...",The result provided is the linear regression c...,"[STUD_MATH, ESCS]","[(STUD_MATH, Student's score in Mathematics), ...","def run(data):\n data = data[['STUD_MATH', ..."
...,...,...,...,...,...,...
103,Can you identify the countries where students ...,None,"Based on the result provided, it appears that ...","[CNT, PV10READ]","[(REGION, REGION), (OECD, OECD country), (COBN...",def run(data):\n result = data.dropna(subse...
104,Which countries rank at the bottom for student...,"([479.5922458857696], Mean)",The lowest 5 countries for student reading sco...,"[CNT, STUD_READ]","[(REGION, REGION), (OECD, OECD country), (COBN...",def run(data):\n result = data.dropna(subse...
105,I'm curious about the countries with the lowes...,"([479.5922458857696], mean)",The bottom 5 countries with the lowest reading...,"[CNT, STUD_READ]","[(REGION, REGION), (OECD, OECD country), (COBN...",def run(data):\n result = data.dropna(subse...
106,Do you know which countries have the lowest re...,None,"Based on the provided result, it appears that ...","[CNT, PV10READ]","[(REGION, REGION), (OECD, OECD country), (COBN...",def run(data):\n result = data.dropna(subse...


In [74]:
pdres.to_csv('result.csv')